In [1]:
import weaviate
import os
import json

wv_url = "https://jptest.weaviate.network"
api_key = os.environ.get("OPENAI_API_KEY")

auth = weaviate.AuthClientPassword(
    username=os.environ.get("WCS_USER"),
    password=os.environ.get("WCS_PASS"),
)

client = weaviate.Client(
    url=wv_url,
    auth_client_secret=auth,
    additional_headers={"X-OpenAI-Api-Key": api_key},
)

In [2]:
client.schema.delete_all()

In [3]:
import wv_datasets

dataset = wv_datasets.JeopardyQuestions()  # Try wv_datasets.WikiArticles or wv_datasets.WineReviews
print(dataset.classes_in_schema(client))
class_defs = dataset.see_class_definitions()
print(json.dumps(class_defs, indent=2))

{'JeopardyCategory': False, 'JeopardyQuestion': False}
[
  {
    "class": "JeopardyCategory",
    "description": "A Jeopardy! category",
    "vectorizer": "text2vec-openai"
  },
  {
    "class": "JeopardyQuestion",
    "description": "A Jeopardy! question",
    "vectorizer": "text2vec-openai",
    "properties": [
      {
        "name": "hasCategory",
        "dataType": [
          "JeopardyCategory"
        ],
        "description": "The category of the question"
      },
      {
        "name": "question",
        "dataType": [
          "text"
        ],
        "description": "Question asked to the contestant"
      },
      {
        "name": "answer",
        "dataType": [
          "text"
        ],
        "description": "Answer provided by the contestant"
      },
      {
        "name": "value",
        "dataType": [
          "int"
        ],
        "description": "Points that the question was worth"
      },
      {
        "name": "round",
        "dataType": [
          

In [4]:
dataset.add_to_schema(client)
dataset.upload_objects(client, batch_size=100)

1000it [00:52, 18.89it/s]


True

In [5]:
client.query.get(dataset.get_class_names()[0], ["title"]).with_limit(5).do()

{'data': {'Get': {'JeopardyCategory': [{'title': 'WORD ORIGINS'},
    {'title': 'LANGUAGES'},
    {'title': 'ANIMALS'},
    {'title': 'ART'},
    {'title': 'AMERICAN HISTORY'}]}}}

In [8]:
client.query.aggregate(dataset.get_class_names()[0]).with_meta_count().do()

{'data': {'Aggregate': {'JeopardyCategory': [{'meta': {'count': 40}}]}}}